<a href="https://colab.research.google.com/github/x1001000/Colab-Notebooks/blob/main/%E5%9F%BA%E6%96%BCGoogle%E8%AA%9E%E9%9F%B3%E8%BE%A8%E8%AD%98%E7%9A%84%E6%97%A5%E8%AA%9E%E7%99%BC%E9%9F%B3%E8%A9%95%E5%AF%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. TEXT可任意增刪，執行即可更新，任選一行實測

In [ ]:
TEXT = '''
白雪姫のお母さんは、白雪姫がまだ子供のころ、重い病気にかかって亡くなってしまいました。
二度目のお母さんはとてもきれいな人でした。けれども、自分よりきれいな人がいることには我慢のできない人でした。
毎朝、不思議な鏡にこうたずねていたのです。「鏡よ、鏡。この世の中で一番美しいのはだれ。」
「この世の中で一番美しいのは、おきさき様あなたです。」鏡はいつもこうこたえていました。
白雪姫のお母さんは、白雪姫がまだ子供のころ、重い病気にかかって亡くなってしまいました。二度目のお母さんはとてもきれいな人でした。けれども、自分よりきれいな人がいることには我慢のできない人でした。毎朝、不思議な鏡にこうたずねていたのです。「鏡よ、鏡。この世の中で一番美しいのはだれ。」「この世の中で一番美しいのは、おきさき様あなたです。」鏡はいつもこうこたえていました。
2004年にお札のデザインが新しくされた。日本のお札が刷新されたのは20年ぶりだ。
'''.split('\n')

## 2. 重複錄音及辨識

👇這格只需執行一次，產生錄音機之後，只需按 ● 即可重複錄音

In [ ]:
!pip install SpeechRecognition >> log.txt
import speech_recognition as sr
recognizer = sr.Recognizer()

!pip install sudachipy >> log.txt
import sudachipy.tokenizer
mode = sudachipy.tokenizer.Tokenizer.SplitMode.C

!pip install sudachidict_core >> log.txt
import sudachipy.dictionary
tokenizer = sudachipy.dictionary.Dictionary().create()

!pip install jaconv >> log.txt
import jaconv

# !pip install pykakasi >> log.txt
# import pykakasi
# kks = pykakasi.kakasi()

from google.colab import output
output.enable_custom_widget_manager()

!pip install ipywebrtc >> log.txt
from ipywebrtc import CameraStream, AudioRecorder
camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

👇每次錄音後執行這格，將音訊存成recording.wav檔，傳送至Google語音辨識API

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav recording.wav -y -hide_banner -loglevel panic

with sr.AudioFile('recording.wav') as source:
    audio = recognizer.record(source)
    print('Google語音辨識API回傳：\n')
    try:
        recognized_text = recognizer.recognize_google(audio, language='ja-JP')
        print(recognized_text)
    except sr.UnknownValueError:
        print('未知錯誤')

Google語音辨識API回傳：

白雪姫のお母さんは白雪姫がまだ子供の頃 重い病気にかかって亡くなってしまいました 白雪姫のお母さんは白雪姫がまだ子供の頃 重い病気にかかって亡くなってしまいました


## 3. 發音準確度（平假名相似度）

In [ ]:
def N(n):
    N = ''
    char = ['', '一', '二', '三', '四', '五', '六', '七', '八', '九']
    for UNIT in ['', '萬', '億', '兆']:
        N += UNIT
        for unit in ['', '十', '百', '千']:
            digit = n % 10
            if digit != 0:
                N += unit
            if unit == '':          # 個位
                N += char[digit]    # 轉漢字接到字串N後
            elif digit != 1:        # 十百千位，數字非1
                N += char[digit]    # 轉漢字接到字串N後
            n //= 10
            if n == 0:
                return N[::-1]      # 反轉字串N

def repl(matchobj):
    return N(int(matchobj.group()))

# @title TEXT轉假名
實測TEXT第幾行 = 1 # @param {type:"integer"}

import re
text = re.sub('[、。「」 ]', '', TEXT[實測TEXT第幾行])
text = re.sub(r'\d+', repl, text)
tokens = tokenizer.tokenize(text, mode)
text_orig = ''.join([token.surface() for token in tokens])
text_kata = ''.join([token.reading_form() for token in tokens])
text_hira = jaconv.kata2hira(text_kata)
# tokens = kks.convert(text)
# text_orig = ''.join([token['orig'] for token in tokens])
# text_hira = ''.join([token['hira'] for token in tokens])
# text_kana = ''.join([token['kana'] for token in tokens])
print(f'TEXT\t{text_orig}')
print(f'片假名\t{text_kata}')
print(f'平假名\t{text_hira}')

TEXT	白雪姫のお母さんは白雪姫がまだ子供のころ重い病気にかかって亡くなってしまいました
片假名	シラユキヒメノオカアサンハシラユキヒメガマダコドモノコロオモイビョウキニカカッテナクナッテシマイマシタ
平假名	しらゆきひめのおかあさんはしらゆきひめがまだこどものころおもいびょうきにかかってなくなってしまいました


### 語音辨識結果轉假名

In [ ]:
recognizedtext = re.sub('[、。「」 ]', '', recognized_text)
recognizedtext = re.sub(r'\d+', repl, recognizedtext)
tokens = tokenizer.tokenize(recognizedtext, mode)
recognizedtext_orig = ''.join([token.surface() for token in tokens])
recognizedtext_kata = ''.join([token.reading_form() for token in tokens])
recognizedtext_hira = jaconv.kata2hira(recognizedtext_kata)
# tokens = kks.convert(recognizedtext)
# recognizedtext_orig = ''.join([token['orig'] for token in tokens])
# recognizedtext_hira = ''.join([token['hira'] for token in tokens])
# recognizedtext_kana = ''.join([token['kana'] for token in tokens])
print(f'語音辨識\t{recognizedtext_orig}')
print(f'片假名\t{recognizedtext_kata}')
print(f'平假名\t{recognizedtext_hira}')

語音辨識	白雪姫のお母さんは白雪姫がまだ子供の頃重い病気にかかって亡くなってしまいました白雪姫のお母さんは白雪姫がまだ子供の頃重い病気にかかって亡くなってしまいました
片假名	シラユキヒメノオカアサンハシラユキヒメガマダコドモノコロオモイビョウキニカカッテナクナッテシマイマシタシラユキヒメノオカアサンハシラユキヒメガマダコドモノコロオモイビョウキニカカッテナクナッテシマイマシタ
平假名	しらゆきひめのおかあさんはしらゆきひめがまだこどものころおもいびょうきにかかってなくなってしまいましたしらゆきひめのおかあさんはしらゆきひめがまだこどものころおもいびょうきにかかってなくなってしまいました


### TEXT平假名與語音辨識結果平假名相似度

In [ ]:
import difflib
similarity = difflib.SequenceMatcher(None, text_hira, recognizedtext_hira).ratio()
print(f'相似度: {similarity}')

相似度: 0.6666666666666666
